<a href="https://colab.research.google.com/github/wonkwonlee/a2w-kma-big-data-contest/blob/main/XGBoost_undersample_submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks')

In [ ]:
import tensorflow as tf
from tensorflow import keras

import os
import tempfile

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression


from xgboost import plot_importance
from xgboost import XGBClassifier

In [ ]:
def metrics(y_test, pred):
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    roc_score = roc_auc_score(y_test, pred)
    print('정확도 : {0:.2f}, 정밀도 : {1:.2f}, 재현율 : {2:.2f}'.format(accuracy, precision, recall))
    print('f1-score : {0:.2f}, auc : {1:.2f}'.format(f1, roc_score))

# 데이터 처리 및 탐색

In [ ]:
raw_df = pd.read_csv("../A2W/데이터/data_submission/merge_final_real.csv", encoding='utf-8-sig')

In [ ]:
raw_df['1day_yn'] = 0.0
raw_df['2day_yn'] = 0.0

In [ ]:
# 열을 밀어서 출력, 맨 첫날, 마지막 날의 경우 산사태가 발생하지 않음.
raw_df['1day_yn'][1:-1] = raw_df['landslide'][0:-2]
raw_df['2day_yn'][2:-1] = raw_df['landslide'][0:-3]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
select_df = raw_df[['maxTa','minTa','avgTa','EIGHT_AGL','stnId','mi10MaxRn', 'hr1MaxRn', 'sumRnDur', 'sumRn', 'maxInsWs', 'maxWs', 'avgWs','minRhm','avgRhm','n99Rn', 'PRRCK_LARG', 'PRRCK_MDDL','LOCTN_ALTT','LOCTN_GRDN','CLZN_CD','TPGRP_TPCD','PRDN_FOM_C','SLDPT_TPCD','SLANT_TYP','SCSTX_CD','SLTP_CD','STORUNST','FROR_CD','FRTP_CD','KOFTR_GROU','DMCLS_CD','AGCLS_CD','HEIGHT','1day_yn']]
cleaned_df_1 = select_df.fillna(0)
cleaned_df_1.info()
cleaned_df_1.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2093790 entries, 0 to 2093789
Data columns (total 34 columns):
 #   Column      Dtype  
---  ------      -----  
 0   maxTa       float64
 1   minTa       float64
 2   avgTa       float64
 3   EIGHT_AGL   float64
 4   stnId       int64  
 5   mi10MaxRn   float64
 6   hr1MaxRn    float64
 7   sumRnDur    float64
 8   sumRn       float64
 9   maxInsWs    float64
 10  maxWs       float64
 11  avgWs       float64
 12  minRhm      float64
 13  avgRhm      float64
 14  n99Rn       float64
 15  PRRCK_LARG  int64  
 16  PRRCK_MDDL  int64  
 17  LOCTN_ALTT  float64
 18  LOCTN_GRDN  float64
 19  CLZN_CD     int64  
 20  TPGRP_TPCD  int64  
 21  PRDN_FOM_C  int64  
 22  SLDPT_TPCD  int64  
 23  SLANT_TYP   int64  
 24  SCSTX_CD    int64  
 25  SLTP_CD     int64  
 26  STORUNST    float64
 27  FROR_CD     float64
 28  FRTP_CD     float64
 29  KOFTR_GROU  float64
 30  DMCLS_CD    float64
 31  AGCLS_CD    float64
 32  HEIGHT      float64
 33  1day_yn

,maxTa,minTa,avgTa,EIGHT_AGL,stnId,mi10MaxRn,hr1MaxRn,sumRnDur,sumRn,maxInsWs,maxWs,avgWs,minRhm,avgRhm,n99Rn,PRRCK_LARG,PRRCK_MDDL,LOCTN_ALTT,LOCTN_GRDN,CLZN_CD,TPGRP_TPCD,PRDN_FOM_C,SLDPT_TPCD,SLANT_TYP,SCSTX_CD,SLTP_CD,STORUNST,FROR_CD,FRTP_CD,KOFTR_GROU,DMCLS_CD,AGCLS_CD,HEIGHT,1day_yn
0,2.1,-1.1,0.8,0.0,115,0.0,0.0,19.33,28.0,29.4,19.0,11.1,65.0,82.5,5.8,0,0,0.0,0.0,0,0,0,0,0,0,0,1.0,2.0,2.0,30.0,1.0,5.0,16.0,0.0
1,1.3,-0.9,0.1,0.0,115,0.0,0.0,22.83,24.8,14.5,8.5,6.0,65.0,91.5,29.7,0,0,0.0,0.0,0,0,0,0,0,0,0,1.0,2.0,2.0,30.0,1.0,5.0,16.0,0.0
2,0.8,-1.4,-0.3,0.0,115,0.0,0.0,12.83,7.5,15.9,10.2,6.5,61.0,80.3,0.3,0,0,0.0,0.0,0,0,0,0,0,0,0,1.0,2.0,2.0,30.0,1.0,5.0,16.0,0.0
3,3.9,0.4,1.7,0.0,115,0.0,0.0,0.00,0.0,11.2,6.8,3.1,53.0,63.1,0.0,0,0,0.0,0.0,0,0,0,0,0,0,0,1.0,2.0,2.0,30.0,1.0,5.0,16.0,0.0
4,2.6,-1.2,1.2,0.0,115,0.0,0.0,7.25,1.8,13.9,6.9,4.1,42.0,61.5,12.3,0,0,0.0,0.0,0,0,0,0,0,0,0,1.0,2.0,2.0,30.0,1.0,5.0,16.0,0.0


## 1일 후 예측

In [ ]:
y = cleaned_df_1['1day_yn']
X = cleaned_df_1.drop('1day_yn', axis =1)

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

# 1: 50 비율
X_samp, y_samp = RandomUnderSampler(random_state=1024, ratio={1: 359, 0: 17950}).fit_sample(X, y)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_samp, y_samp, test_size=0.25, shuffle=True, stratify=y_samp, random_state=1024)

In [ ]:
X_train

array([[24.8, 11. , 19. , ...,  2. ,  4. , 12. ],
       [ 8.6, -0.4,  3.4, ...,  1. ,  4. , 10. ],
       [13.3, -5.7,  4. , ...,  2. ,  5. , 12. ],
       ...,
       [20.7, 11.1, 14.7, ...,  2. ,  4. , 12. ],
       [24.9, 16.7, 20.8, ...,  2. ,  5. , 14. ],
       [ 8.4, -1. ,  2.1, ...,  2. ,  5. , 16. ]])

### 하이퍼 파라미터 튜닝 후 예측

In [ ]:
xgb = XGBClassifier()

xgb_param_grid = {
    'n_estimators' : [100, 200, 400, 500, 600],
    'learning_rate' : [0.01, 0.05, 0.1, 0.15, 0.2],
    'max_depth' : [4, 6, 8, 10, 12],
}
# sklearn.metrics.SCORERS.keys()
xgb_grid = GridSearchCV(xgb, param_grid = xgb_param_grid, scoring="roc_auc", n_jobs= 1, verbose = 1)
xgb_grid.fit(X_train, y_train.values.ravel())

In [ ]:
print("최고 평균 auc : {0:.4f}".format(xgb_grid.best_score_))
print("최고의 파라미터 : ", xgb_grid.best_params_)
print("최고 점수를 낸 파라미터를 가진 모형 : ", xgb_grid.best_estimator_)

In [ ]:
result = pd.DataFrame(xgb_grid.cv_results_)
result.sort_values(by=['rank_test_score'], inplace=True)

In [ ]:
result[['params', 'mean_test_score', 'rank_test_score']].head(10)

In [ ]:
import joblib

xgb1 = xgb_grid.best_estimator_
xgb1.fit(X_train, y_train.values.ravel())
joblib.dump(xgb1, './xgb_0623_model_1day_yn.pkl')

['./xgb_0623_model_1day_yn.pkl']

In [ ]:
xgb1 = XGBClassifier(n_estimators=200, learning_rate = 0.05, max_depth = 10)
evals = [(X_test, y_test)]
xgb1.fit(X_train, y_train, eval_metric = "logloss", eval_set = evals, verbose = 1)

In [ ]:
xgb1_pred = xgb1.predict(X_test)
metrics(y_test, xgb1_pred)

정확도 : 0.98, 정밀도 : 0.00, 재현율 : 0.00
f1-score : 0.00, auc : 0.50


In [ ]:
import pickle

with open('xgb1_model.pkl', 'wb') as model_file:
  pickle.dump(xgb1, model_file)

In [ ]:
# Merged_data_file
Merged_2020 = pd.read_csv('../A2W/데이터/data_submission/merge_s_h_a20.csv')
# Test 검증 데이터셋
test_input = pd.read_csv('../A2W/데이터/data_submission/input.csv', encoding = 'cp949')

In [ ]:
length_test_input = len(test_input.index)

# for index in range(1):
input_date = test_input['date'][1]
input_sgg = test_input['sgg'][1]
input_umd = test_input['umd'][1]

text = input_sgg + ' ' + input_umd
# Merged_2020 = Merged_2020.to_numpy()

test_day1 = Merged_2020[Merged_2020['umd2'].str.contains(text.rstrip()) & Merged_2020['tm'].str.contains(input_date)]
test_specific_day1 = test_day1[['maxTa','minTa','avgTa','EIGHT_AGL','stnId','mi10MaxRn','hr1MaxRn','sumRnDur',
                                    'sumRn','maxInsWs','maxWs','avgWs','minRhm','avgRhm','n99Rn','PRRCK_LARG',
                                    'PRRCK_MDDL','LOCTN_ALTT','LOCTN_GRDN','CLZN_CD','TPGRP_TPCD','PRDN_FOM_C',
                                    'SLDPT_TPCD','SLANT_TYP','SCSTX_CD','SLTP_CD','STORUNST','FROR_CD','FRTP_CD',
                                    'KOFTR_GROU','DMCLS_CD','AGCLS_CD','HEIGHT']]

# test_vector = test_specific_day1[['PRDN_FOM_C','maxInsWs','maxTa','TPGRP_TPCD','hr1MaxRn','minRhm','SLANT_TYP',
#                                   'PRRCK_MDDL','maxWs','FRTP_CD','sumRnDur','SLDPT_TPCD','DMCLS_CD','n99Rn',
#                                   'LOCTN_GRDN','avgRhm','STORUNST','sumRn','EIGHT_AGL','FROR_CD','KOFTR_GROU',
#                                   'CLZN_CD','stnId','HEIGHT','avgTa','SCSTX_CD','SLTP_CD','minTa','PRRCK_LARG', 
#                                   'LOCTN_ALTT','avgWs','mi10MaxRn','AGCLS_CD']]

# dic = {'PRDN_FOM_C':'27','maxInsWs':'14','maxTa':'20','TPGRP_TPCD':'4 ','hr1MaxRn':'1 ','minRhm':'23','SLANT_TYP':'10',
#        'PRRCK_MDDL':'24','maxWs':'9 ','FRTP_CD':'11','sumRnDur':'8 ','SLDPT_TPCD':'5 ','DMCLS_CD':'15','n99Rn':'21',
#        'LOCTN_GRDN':'13','avgRhm':'2 ','STORUNST':'19','sumRn':'29','EIGHT_AGL':'3 ','FROR_CD':'32','KOFTR_GROU':'22',
#        'CLZN_CD':'12','stnId':'30','HEIGHT':'31','avgTa':'17','SCSTX_CD':'6 ','SLTP_CD':'26','minTa':'16','PRRCK_LARG':'25', 
#        'LOCTN_ALTT':'7 ','avgWs':'18','mi10MaxRn':'0 ','AGCLS_CD':'28'}

# test_vector = test_vector.rename(columns=dic)

# test_vector
# test_specific_day1 = test_specific_day1[xgb1.get_booster().feature_names]
model_pred = xgb1.predict(test_specific_day1.to_numpy())
model_pred
# test_input['1day_yn'][1] = model_pred

array([0.])

In [ ]:
# 훈련된 모델로 결과 도출 : day1 함수
def day1_data_test(sgg, umd, date):    
    # sgg 데이터와 umd 데이터, date를 통한 추출
    text = sgg + ' ' + umd
    # specific 자체가 학습 돌린 결과 참고해서 input으로 만들어야 됨
    test_day1 = Merged_2020[Merged_2020['umd2'].str.contains(text.rstrip()) & Merged_2020['tm'].str.contains(date)]
    test_specific_day1 = test_day1[['maxTa','minTa','avgTa','EIGHT_AGL','stnId','mi10MaxRn', 'hr1MaxRn', 'sumRnDur', 'sumRn', 'maxInsWs', 
                                    'maxWs', 'avgWs','minRhm','avgRhm','n99Rn', 'PRRCK_LARG', 'PRRCK_MDDL','LOCTN_ALTT','LOCTN_GRDN',
                                    'CLZN_CD','TPGRP_TPCD','PRDN_FOM_C','SLDPT_TPCD','SLANT_TYP','SCSTX_CD','SLTP_CD','STORUNST','FROR_CD',
                                    'FRTP_CD','KOFTR_GROU','DMCLS_CD','AGCLS_CD','HEIGHT']]
    fillna_test_specific_day1 = test_specific_day1.fillna(0)
    model_pred = xgb1.predict(fillna_test_specific_day1.to_numpy())
    return model_pred

In [ ]:
length_test_input = len(test_input.index)

#day_1 data
for index in range(length_test_input):
    if test_input['umd'][index] == "삼거동":
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = "상문동"
    elif test_input['umd'][index] == "어곡동":
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = "강서동"
    elif test_input['umd'][index] == "주진동":
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = "소주동"
    elif test_input['umd'][index] == "시동":
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = "불국동"
    elif test_input['umd'][index] == "덕곡동":
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = "지좌동"
    elif test_input['umd'][index] == "조와동":
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = "상망동"
    else:
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = test_input['umd'][index]
    test_input['1day_yn'][index] = day1_data_test(input_sgg, input_umd, input_date)


test_input.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,date,sd,sgg,umd,1day,1day_yn,2day,2day_yn
0,2020-06-10,경상남도,사천시,곤명면,2020-06-11,0.0,2020-06-12,NaN
1,2020-06-11,경상남도,사천시,곤명면,2020-06-12,0.0,2020-06-13,NaN
2,2020-06-12,경상남도,사천시,곤명면,2020-06-13,0.0,2020-06-14,NaN
3,2020-06-10,경상남도,사천시,곤양면,2020-06-11,0.0,2020-06-12,NaN
4,2020-06-11,경상남도,사천시,곤양면,2020-06-12,0.0,2020-06-13,NaN


In [ ]:
test_input['1day_yn'].value_counts()

0.0    3384
1.0      26
Name: 1day_yn, dtype: int64

## 2일 후 예측

In [ ]:
#분석을 위해 음수 값을 가지는 칼럼들을 제외하고 결측치를 0으로 대체한다.
select_df = raw_df[['maxTa','minTa','avgTa','EIGHT_AGL','stnId','mi10MaxRn', 'hr1MaxRn', 'sumRnDur', 'sumRn', 'maxInsWs', 'maxWs', 'avgWs','minRhm','avgRhm','n99Rn', 'PRRCK_LARG', 'PRRCK_MDDL','LOCTN_ALTT','LOCTN_GRDN','CLZN_CD','TPGRP_TPCD','PRDN_FOM_C','SLDPT_TPCD','SLANT_TYP','SCSTX_CD','SLTP_CD','STORUNST','FROR_CD','FRTP_CD','KOFTR_GROU','DMCLS_CD','AGCLS_CD','HEIGHT','2day_yn']]
cleaned_df_2 = select_df.fillna(0)

In [ ]:
y = cleaned_df_2['2day_yn']
X = cleaned_df_2.drop('2day_yn', axis =1)

In [ ]:
# 1: 50 비율
X_samp, y_samp = RandomUnderSampler(random_state=1024, ratio={1: 359, 0: 17950}).fit_sample(X, y)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_samp, y_samp, test_size=0.25, shuffle=True, stratify=y_samp, random_state=1024)

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

X_train shape: (13731, 33)
X_test shape: (4578, 33)
y_train shape: (13731,)
y_test shape: (4578,)


In [ ]:
xgb = XGBClassifier()

xgb_param_grid = {
    'n_estimators' : [100, 200, 400, 500, 600],
    'learning_rate' : [0.01, 0.05, 0.1, 0.15, 0.2],
    'max_depth' : [4, 6, 8, 10, 12],
}
# sklearn.metrics.SCORERS.keys()
xgb_grid = GridSearchCV(xgb, param_grid = xgb_param_grid, scoring="roc_auc", n_jobs= 1, verbose = 1)
xgb_grid.fit(X_train, y_train.values.ravel())

In [ ]:
print("최고 평균 auc : {0:.4f}".format(xgb_grid.best_score_))
print("최고의 파라미터 : ", xgb_grid.best_params_)
print("최고 점수를 낸 파라미터를 가진 모형 : ", xgb_grid.best_estimator_)

In [ ]:
result = pd.DataFrame(xgb_grid.cv_results_)
result.sort_values(by=['rank_test_score'], inplace=True)

In [ ]:
result[['params', 'mean_test_score', 'rank_test_score']].head(10)

In [ ]:
import joblib

xgb2 = xgb_grid.best_estimator_
xgb2.fit(X_train, y_train)
joblib.dump(xgb2, './xgb_0623_model_2day_yn.pkl')

In [ ]:
xgb2_pred = xgb2.predict(X_test)
metrics(y_test, xgb2_pred)

In [ ]:
xgb2 = XGBClassifier(n_estimators=400, learning_rate = 0.05, max_depth = 10)
evals = [(X_test, y_test)]
xgb2.fit(X_train, y_train, early_stopping_rounds = 100, eval_metric = "logloss", eval_set = evals, verbose = 1)

[0]	validation_0-logloss:0.646338
Will train until validation_0-logloss hasn't improved in 100 rounds.
[1]	validation_0-logloss:0.604198
[2]	validation_0-logloss:0.565844
[3]	validation_0-logloss:0.530598
[4]	validation_0-logloss:0.498472
[5]	validation_0-logloss:0.468688
[6]	validation_0-logloss:0.441466
[7]	validation_0-logloss:0.416138
[8]	validation_0-logloss:0.392789
[9]	validation_0-logloss:0.371071
[10]	validation_0-logloss:0.350803
[11]	validation_0-logloss:0.33199
[12]	validation_0-logloss:0.314395
[13]	validation_0-logloss:0.297966
[14]	validation_0-logloss:0.282751
[15]	validation_0-logloss:0.268444
[16]	validation_0-logloss:0.254991
[17]	validation_0-logloss:0.24215
[18]	validation_0-logloss:0.230079
[19]	validation_0-logloss:0.21899
[20]	validation_0-logloss:0.208608
[21]	validation_0-logloss:0.198886
[22]	validation_0-logloss:0.189501
[23]	validation_0-logloss:0.18077
[24]	validation_0-logloss:0.172639
[25]	validation_0-logloss:0.165007
[26]	validation_0-logloss:0.157604


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.05, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=None, n_estimators=400, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
xgb2_pred = xgb2.predict(X_test)
metrics(y_test, xgb2_pred)

정확도 : 1.00, 정밀도 : 0.97, 재현율 : 0.78
f1-score : 0.86, auc : 0.89


# Submission 실행

In [ ]:
# Merged_data_file
Merged_2020 = pd.read_csv('../A2W/데이터/data_submission/merge_s_h_a20.csv')
# Test 검증 데이터셋
test_input = pd.read_csv('../A2W/데이터/data_submission/input.csv', encoding = 'cp949')

In [ ]:
# 훈련된 모델로 결과 도출 : day1 함수
def day1_data_test(sgg, umd, date):    
    # sgg 데이터와 umd 데이터, date를 통한 추출
    text = sgg + ' ' + umd
    # specific 자체가 학습 돌린 결과 참고해서 input으로 만들어야 됨
    test_day1 = Merged_2020[Merged_2020['umd2'].str.contains(text.rstrip()) & Merged_2020['tm'].str.contains(date)]
    test_specific_day1 = test_day1[['maxTa','minTa','avgTa','EIGHT_AGL','stnId','mi10MaxRn', 'hr1MaxRn', 'sumRnDur', 'sumRn', 'maxInsWs', 
                                    'maxWs', 'avgWs','minRhm','avgRhm','n99Rn', 'PRRCK_LARG', 'PRRCK_MDDL','LOCTN_ALTT','LOCTN_GRDN',
                                    'CLZN_CD','TPGRP_TPCD','PRDN_FOM_C','SLDPT_TPCD','SLANT_TYP','SCSTX_CD','SLTP_CD','STORUNST','FROR_CD',
                                    'FRTP_CD','KOFTR_GROU','DMCLS_CD','AGCLS_CD','HEIGHT']]
    fillna_test_specific_day1 = test_specific_day1.fillna(0)
    model_pred = xgb1.predict(fillna_test_specific_day1.to_numpy())
    return model_pred

# 훈련된 모델로 결과 도출 : day2 함수
def day2_data_test(sgg, umd, date):    
    # sgg 데이터와 umd 데이터, date를 통한 추출
    text = sgg + ' ' + umd
    # specific 자체가 학습 돌린 결과 참고해서 input으로 만들어야 됨
    test_day2 = Merged_2020[Merged_2020['umd2'].str.contains(text.rstrip()) & Merged_2020['tm'].str.contains(date)]
    test_specific_day2 = test_day2[['maxTa','minTa','avgTa','EIGHT_AGL','stnId','mi10MaxRn', 'hr1MaxRn', 'sumRnDur', 'sumRn', 'maxInsWs',
                                    'maxWs', 'avgWs','minRhm','avgRhm','n99Rn', 'PRRCK_LARG', 'PRRCK_MDDL','LOCTN_ALTT','LOCTN_GRDN',
                                    'CLZN_CD','TPGRP_TPCD','PRDN_FOM_C','SLDPT_TPCD','SLANT_TYP','SCSTX_CD','SLTP_CD','STORUNST','FROR_CD',
                                    'FRTP_CD','KOFTR_GROU','DMCLS_CD','AGCLS_CD','HEIGHT']]
    fillna_test_specific_day2 = test_specific_day2.fillna(0)
    model_pred = xgb2.predict(fillna_test_specific_day2.to_numpy())
    return model_pred

In [ ]:
length_test_input = len(test_input.index)

# #day_1 data
# for index in range(length_test_input):
#     if test_input['umd'][index] == "삼거동":
#         input_date = test_input['date'][index]
#         input_sgg = test_input['sgg'][index]
#         input_umd = "상문동"
#     elif test_input['umd'][index] == "어곡동":
#         input_date = test_input['date'][index]
#         input_sgg = test_input['sgg'][index]
#         input_umd = "강서동"
#     elif test_input['umd'][index] == "주진동":
#         input_date = test_input['date'][index]
#         input_sgg = test_input['sgg'][index]
#         input_umd = "소주동"
#     elif test_input['umd'][index] == "시동":
#         input_date = test_input['date'][index]
#         input_sgg = test_input['sgg'][index]
#         input_umd = "불국동"
#     elif test_input['umd'][index] == "덕곡동":
#         input_date = test_input['date'][index]
#         input_sgg = test_input['sgg'][index]
#         input_umd = "지좌동"
#     elif test_input['umd'][index] == "조와동":
#         input_date = test_input['date'][index]
#         input_sgg = test_input['sgg'][index]
#         input_umd = "상망동"
#     else:
#         input_date = test_input['date'][index]
#         input_sgg = test_input['sgg'][index]
#         input_umd = test_input['umd'][index]
#     test_input['1day_yn'][index] = day1_data_test(input_sgg, input_umd, input_date)
        

#day_2 data
for index in range(length_test_input):
    if test_input['umd'][index] == "삼거동":
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = "상문동"
    elif test_input['umd'][index] == "어곡동":
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = "강서동"
    elif test_input['umd'][index] == "주진동":
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = "소주동"
    elif test_input['umd'][index] == "시동":
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = "불국동"
    elif test_input['umd'][index] == "덕곡동":
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = "지좌동"
    elif test_input['umd'][index] == "조와동":
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = "상망동"
    else:
        input_date = test_input['date'][index]
        input_sgg = test_input['sgg'][index]
        input_umd = test_input['umd'][index]
    test_input['2day_yn'][index] = day2_data_test(input_sgg, input_umd, input_date)
        
# print(search_data(input_sgg, input_umd, input_date))


test_input.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,date,sd,sgg,umd,1day,1day_yn,2day,2day_yn
0,2020-06-10,경상남도,사천시,곤명면,2020-06-11,0.0,2020-06-12,0.0
1,2020-06-11,경상남도,사천시,곤명면,2020-06-12,0.0,2020-06-13,0.0
2,2020-06-12,경상남도,사천시,곤명면,2020-06-13,0.0,2020-06-14,0.0
3,2020-06-10,경상남도,사천시,곤양면,2020-06-11,0.0,2020-06-12,0.0
4,2020-06-11,경상남도,사천시,곤양면,2020-06-12,0.0,2020-06-13,0.0


In [ ]:
test_input["1day_yn"].value_counts()

0.0    3384
1.0      26
Name: 1day_yn, dtype: int64

In [ ]:
test_input["2day_yn"].value_counts()

0.0    3410
Name: 2day_yn, dtype: int64

In [ ]:
test_input.to_csv("xgb_test_model.csv", index=False, encoding='cp949')